# 📊 **Customer Care Agent - KPI Dashboard & Performance Analytics**

## 🎯 **Overview**
This notebook provides comprehensive analytics and performance metrics for the Customer Care Agent system. It reads data from `Matrix.csv` and provides real-time insights into system performance.

### 📈 **Key Features**
- **Real-time KPI Monitoring**: Live metrics from evaluation results
- **Performance Trends**: Historical performance analysis
- **Scenario Analysis**: Breakdown by test types and personas
- **LLM Judge Scores**: AI-powered evaluation metrics
- **Interactive Visualizations**: Charts and graphs for insights

### 🔧 **Setup Instructions**
1. Run the `Master_Customer_Care_Agent_LLM.ipynb` to generate evaluation data
2. This notebook will automatically read the `Matrix.csv` file
3. All charts and metrics will update in real-time

---


In [1]:
# 📦 Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 KPI Dashboard Libraries Loaded Successfully!")
print("🎨 Visualization styles configured")
print("📈 Ready for analytics and reporting")


📊 KPI Dashboard Libraries Loaded Successfully!
🎨 Visualization styles configured
📈 Ready for analytics and reporting


In [2]:
# 📊 Data Loading and Preprocessing
import os
import json
from datetime import datetime

def load_evaluation_data():
    """Load and preprocess evaluation data from Matrix.csv"""
    matrix_file = "Matrix.csv"
    
    if not os.path.exists(matrix_file):
        print("❌ Matrix.csv not found!")
        print("💡 Please run the Master_Customer_Care_Agent_LLM.ipynb first to generate evaluation data")
        return None
    
    try:
        # Load the CSV data
        df = pd.read_csv(matrix_file)
        
        # Convert timestamp to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Parse follow-up questions from JSON
        df['follow_up_questions'] = df['follow_up_questions'].apply(
            lambda x: json.loads(x) if x and x != '' else []
        )
        
        # Calculate derived metrics
        df['follow_up_count'] = df['follow_up_questions'].apply(len)
        df['has_follow_ups'] = df['follow_up_count'] > 0
        
        # Convert boolean columns
        bool_columns = ['auth_required', 'auth_successful', 'human_handover', 'resolution_achieved', 'error_occurred']
        for col in bool_columns:
            if col in df.columns:
                df[col] = df[col].astype(bool)
        
        # Calculate overall performance score
        score_columns = [
            'intent_classification_accuracy_score',
            'response_relevance_score', 
            'auth_decision_correctness_score',
            'handover_appropriateness_score',
            'follow_up_quality_score',
            'overall_satisfaction_score'
        ]
        
        df['overall_performance_score'] = df[score_columns].mean(axis=1)
        
        print(f"✅ Data loaded successfully!")
        print(f"📊 Total records: {len(df)}")
        print(f"📅 Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
        print(f"🧪 Test types: {df['scenario_type'].nunique()} unique types")
        print(f"👥 Personas: {df['persona'].nunique()} unique personas")
        
        return df
        
    except Exception as e:
        print(f"❌ Error loading data: {e}")
        return None

# Load the data
df = load_evaluation_data()


❌ Error loading data: the JSON object must be str, bytes or bytearray, not float


In [3]:
# 📊 Key Performance Indicators (KPIs) Summary
if df is not None:
    print("🎯 KEY PERFORMANCE INDICATORS (KPIs)")
    print("="*60)
    
    # Overall Performance Metrics
    total_tests = len(df)
    successful_tests = len(df[~df['error_occurred']])
    success_rate = (successful_tests / total_tests * 100) if total_tests > 0 else 0
    
    # Authentication Metrics
    auth_required_tests = len(df[df['auth_required']])
    auth_successful_tests = len(df[df['auth_successful']])
    auth_success_rate = (auth_successful_tests / auth_required_tests * 100) if auth_required_tests > 0 else 0
    
    # Human Handover Metrics
    handover_tests = len(df[df['human_handover']])
    handover_rate = (handover_tests / total_tests * 100) if total_tests > 0 else 0
    
    # Resolution Metrics
    resolved_tests = len(df[df['resolution_achieved']])
    resolution_rate = (resolved_tests / total_tests * 100) if total_tests > 0 else 0
    
    # Performance Scores
    avg_overall_score = df['overall_performance_score'].mean()
    avg_response_time = df['response_time'].mean()
    
    print(f"📊 OVERALL PERFORMANCE")
    print(f"   Total Tests: {total_tests}")
    print(f"   Success Rate: {success_rate:.1f}%")
    print(f"   Average Performance Score: {avg_overall_score:.2f}/10")
    print(f"   Average Response Time: {avg_response_time:.2f}s")
    
    print(f"\n🔐 AUTHENTICATION METRICS")
    print(f"   Tests Requiring Auth: {auth_required_tests}")
    print(f"   Auth Success Rate: {auth_success_rate:.1f}%")
    
    print(f"\n👨‍💼 HUMAN HANDOVER METRICS")
    print(f"   Handover Rate: {handover_rate:.1f}%")
    print(f"   Resolution Rate: {resolution_rate:.1f}%")
    
    # Score Breakdown
    score_columns = [
        'intent_classification_accuracy_score',
        'response_relevance_score',
        'auth_decision_correctness_score', 
        'handover_appropriateness_score',
        'follow_up_quality_score',
        'overall_satisfaction_score'
    ]
    
    print(f"\n📈 SCORE BREAKDOWN (LLM Judge)")
    for col in score_columns:
        if col in df.columns:
            avg_score = df[col].mean()
            metric_name = col.replace('_score', '').replace('_', ' ').title()
            print(f"   {metric_name}: {avg_score:.2f}/10")
    
else:
    print("❌ No data available for KPI analysis")
    print("💡 Please run evaluations first using Master_Customer_Care_Agent_LLM.ipynb")


❌ No data available for KPI analysis
💡 Please run evaluations first using Master_Customer_Care_Agent_LLM.ipynb


In [4]:
# 📊 Performance Trends Over Time
if df is not None and len(df) > 0:
    # Create time-based analysis
    df['hour'] = df['timestamp'].dt.hour
    df['date'] = df['timestamp'].dt.date
    
    # Plot performance trends
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('📈 Performance Trends Analysis', fontsize=16, fontweight='bold')
    
    # 1. Overall Performance Score Over Time
    daily_scores = df.groupby('date')['overall_performance_score'].mean()
    axes[0,0].plot(daily_scores.index, daily_scores.values, marker='o', linewidth=2)
    axes[0,0].set_title('Overall Performance Score Trend')
    axes[0,0].set_ylabel('Average Score (1-10)')
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Response Time Distribution
    axes[0,1].hist(df['response_time'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0,1].set_title('Response Time Distribution')
    axes[0,1].set_xlabel('Response Time (seconds)')
    axes[0,1].set_ylabel('Frequency')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Success Rate by Scenario Type
    scenario_success = df.groupby('scenario_type').agg({
        'error_occurred': lambda x: (1 - x.mean()) * 100,
        'overall_performance_score': 'mean'
    })
    scenario_success['error_occurred'].plot(kind='bar', ax=axes[1,0], color='lightgreen')
    axes[1,0].set_title('Success Rate by Scenario Type')
    axes[1,0].set_ylabel('Success Rate (%)')
    axes[1,0].tick_params(axis='x', rotation=45)
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Authentication vs Handover Analysis
    auth_handover = df.groupby(['auth_required', 'human_handover']).size().unstack(fill_value=0)
    auth_handover.plot(kind='bar', ax=axes[1,1], stacked=True)
    axes[1,1].set_title('Authentication vs Human Handover')
    axes[1,1].set_ylabel('Number of Tests')
    axes[1,1].tick_params(axis='x', rotation=0)
    axes[1,1].legend(['No Handover', 'Handover'])
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Performance trends visualized successfully!")
else:
    print("❌ No data available for trend analysis")


❌ No data available for trend analysis


In [5]:
# 🔄 Refresh Data and Re-run Analysis
def refresh_data():
    """Refresh data from Matrix.csv and re-run analysis"""
    global df
    print("🔄 Refreshing data from Matrix.csv...")
    df = load_evaluation_data()
    
    if df is not None:
        print("✅ Data refreshed successfully!")
        print(f"📊 Current total records: {len(df)}")
        print(f"📅 Latest timestamp: {df['timestamp'].max()}")
        return True
    else:
        print("❌ Failed to refresh data")
        return False

# Add refresh button functionality
print("🔄 Data Refresh Function Available")
print("💡 Run refresh_data() to update with latest evaluation results")
print("📊 This will automatically update all charts and metrics above")

# Show current data status
if df is not None:
    print(f"\n📊 Current Data Status:")
    print(f"   Records: {len(df)}")
    print(f"   Last Updated: {df['timestamp'].max()}")
    print(f"   Test Types: {df['scenario_type'].nunique()}")
    print(f"   Success Rate: {(1-df['error_occurred'].mean())*100:.1f}%")
else:
    print("\n❌ No data currently loaded")
    print("💡 Run evaluations using Master_Customer_Care_Agent_LLM.ipynb first")


🔄 Data Refresh Function Available
💡 Run refresh_data() to update with latest evaluation results
📊 This will automatically update all charts and metrics above

❌ No data currently loaded
💡 Run evaluations using Master_Customer_Care_Agent_LLM.ipynb first
